<a href="https://colab.research.google.com/github/drbarak/DS_Project1/blob/main/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#About this compare_and_search() program

It allows to compare 2 folders, and optionally thier subfolders, file by file and it displays whether a file is:

1. missing in one of the folders(or subfolders)
2. not identical (size or time modified)
3. identical

The purpose is to check wether a vopy of a folder was successful or to point out differences between a source folder and a target folder whcich is maintained as a backup of the source folder.

The results are displayed on the screen and are saved in a text file name 'compare.txt', so the user can use it to copy, delete or check why there are differences.

Before the program starts the process, it asks the user to enter the source folder and the target folder, and for verification it shows the up to 5 files and up to 5 subfolders in each folder entered by the user.

It then allows the user to specify a filter (mask) so only certain files will be included in the process, to specify wether to include subfolders and wether to dispaly on the screen the names of identical files or only non identical files.

The program also allows just to search for files, by leaving the second folder name ampty (simply to press ENTER)

## Preparing test data
When the program is used in Colab, there is no really data to test.

The project include a section that creates test data, randomly, in folders named 'folder1' and 'folder2'.

To run it, uncomment the last line in the section 'Create test data' which calls the function create_test_data() and then select Run All from the Runtime menu.

Don't forget to comment it again, otherwise new data will be created each time Run All is run

## To run the program
First uncomment the call to compare_or_search() in the section Main Program

Then select Run All from the Runtime menu

#Import libs

In [ ]:
from pathlib import Path
import os
import datetime
from IPython.display import Audio

try:
  from colorama import Fore, Back, Style
except:
  !pip install colorama
  from colorama import Fore, Back, Style

# Part 1

Utility functions

In [ ]:
RED, GREEN = 'red', 'green'

def display_err(message):
  print(Back.YELLOW + Fore.BLACK + message + Style.RESET_ALL)
  sound_file = 'C:/Windows/Media/notify.wav'
  if Path(sound_file).exists():
    Audio(sound_file, autoplay=True)

def get_folder(prompt):
    while True:
        f = input(prompt)
        if f == "":
          return ""
        if f != "":
            f_p = Path(f)
            try:
                if f_p.exists():
                    return f
                  # on Colab if f_p hss wild card it does not raise an exception
                raise exception                    
            except:
                if '*' in f or '?' in f:  # for search allow wild cards
                    return f
        display_err("File or Folder does not exist !!!. try again")
        
def get_YN(prompt):
    while True:
        ans = input(prompt)
        if ans == '':
            ans = 'Y'
        if ans in list('YNyn'):
            return ans.upper() == 'Y'
        display_err("Wrong answer !!!. try again") 
        
def print_files(p, title = '', also_dir = True):
    num = 5
    if title != '':
        print()
        print(f"Files in {title} folder (first {num}):")
    for i, item in enumerate(p.glob('*')):
        if not item.is_dir():
          print('\t',item)
          if i == num - 1:
            break
        
    if also_dir and p.is_dir():
        print()
        print(f"Folders in {title} folder (first {num}):")
        i = 0
        for item in p.iterdir():
            if item.is_dir():
                print('\t', item)
                if i == num - 1:
                    break
                i += 1
        print()

# Part 2

Get info from user

In [ ]:
def set_path(left):
  if left == '.' or left == '.\\' or left == './':
    left_p = Path.cwd()
  elif left == '..' or left == '..\\' or left == '../':
    left_p = Path.cwd().parent
  elif '*' in left or '?' in left:
    return None
  else:
    try:
        left_p = Path(left)
    except:
        return None
  return left_p

def get_info_from_user():
  while True:
      left = get_folder("Enter first File or Folder (leave empty to exit): ")
      if left == "":
        return None, None, None, None, None
      left_p = set_path(left)
      if left_p:
          print_files(left_p, "first")
      
      right = get_folder("Enter second File or Folder (or leave empty to do a search only): ")
      if right == "":
        right_p = None
      elif right == left:
        display_err("Cannot have same folders !!!. Try again")
        continue
      else:
        right_p = set_path(right)
        if right_p:
            print_files(right_p, "second")

      filter = ''
      if left_p == None:  # doing search
            left_p = Path(left)
            left_parts = left_p.parts
            if len(left_parts) == 1: # only file name
                filter = left
                left_p = Path.cwd()
            else:
                filter = left_p.name
                left_p = left_p.parent
            try:
              for item in left_p.glob(filter):
                dummy = item
              break # if reached here then the filter is valid
            except:
                display_err("Invalid file name !!!. Try again")
                continue
      if left_p.is_dir():
          if right == "":
              display_err("Cannot search for a folder !!!. Try again")
              continue
          if right != "" and not right_p.is_dir():
              display_err("Cannot compare folder with a file !!!. Try again")
              continue
          break
      if right != "" and right_p.is_dir():
          display_err("Cannot compare folder with a file !!!. Try again")
          continue
      break

  if filter == '':
      while True:
        filter = input("Enter Filter (press ENTER for ALL [*]) :")
        if filter == "":
          filter = '*'
          break
        try:
          for item in left_p.glob(filter):
            dummy = item
          break # if reached here then the filter is valid
        except:
          display_err(r"Invalid filter (cannot include ':', '\', '/') !!!. Try again")
          
  sub = False
  if '*' in left or left_p.is_dir():    # in case doing search with wild card (left_p)
    sub = get_YN("Include subdirectories (Y=Yes, N = No, press Enter for Yes): ")
  
  identical = False
  if right_p:  # valid for compare only
      identical = get_YN("Show idewntical files (Y=Yes, N = No, press Enter for Yes): ")

  return left_p, right_p, filter, sub, identical

# Part 3

Process dir and subdir

In [ ]:
def comp_file(item, right_item, f, identical):
    if not right_item.exists():
        name = 'folder' if item.is_dir() else 'file'
        my_print(True, RED, f'Left {name} [{item}] does not exist in [{right_item.parent}]', file = f)
        return
    elif item.is_dir() or identical == 2:  # identical = 2 when checking for existance only
        return
    if os.stat(item).st_size != os.stat(right_item).st_size:
        my_print(True, GREEN, f'Left file [{item}] not identical: size = {os.stat(item).st_size}, right size = {os.stat(right_item).st_size}', file = f)
    elif os.stat(item).st_size != os.stat(right_item).st_size:
        my_print(True, GREEN, f'Left file [{item}] not identical: size = {os.stat(item).st_size}, right size = {os.stat(right_item).st_size}', file = f)
    elif os.stat(item).st_mtime != os.stat(right_item).st_mtime:
        left_date = datetime.datetime.fromtimestamp(os.stat(item).st_mtime).strftime("%d/%m/%Y %H:%M:%S")
        right_date = datetime.datetime.fromtimestamp(os.stat(right_item).st_mtime).strftime("%d/%m/%Y %H:%M:%S")
        my_print(True, GREEN, f'Left file [{item}] not identical: date = {left_date}, right date = {right_date}', file = f)
    else:
        my_print(identical, None, f'Left file {item} is identical', file = f)

def my_print(both, color, *args, **kwargs):
  if DEBUG or both:
    if color:
      if color == RED:
        print(Fore.RED, end = '')
      else:
        print(Fore.GREEN, end = '')
      print(*args, end = '')
      print(Style.RESET_ALL)
    else:
      print(*args)
  if not DEBUG:
    print(*args, **kwargs)
    
def dispaly_dir_name(new_left, new_right, count, f):
    if not DEBUG:
        if new_right:
          my_print(False, None, '\tleft dir: [{}\tright dir: [{}'.format(str(new_left)+']', str(new_right)+']'), file = f)
        else:
          my_print(False, None, '\tleft dir: [{}'.format(str(new_left)+']'), file = f)
    count += 1
    if new_right:
        print('{}\tleft dir: [{:<50}\tright dir: [{:<50}'.format(count, str(new_left)[-50:]+']', str(new_right)[-50:]+']'))
    else:
        print('\tleft dir: [{:<50}'.format(str(new_left)[-50:]+']'))
    return count

def process_dir(left_p, right_p, filter, f, count, identical):
    for item in left_p.glob(filter):
        fname = item.name
        if right_p:
          right_item = right_p / fname
          comp_file(item, right_item, f, identical)
          count += 1
          print(count, end = '\r')
        else:
            my_print(identical, None, f'{item}', file = f)
            count += 1
        if count >= MAX:
            break
    return count
    
def process_subdir(left_p, right_p, filter, f, count, identical):
    for item in left_p.iterdir():
        if item.is_dir():
            new_left = left_p / item.name
            if right_p:
                new_right = right_p / item.name
            else:
                new_right = None
            count = dispaly_dir_name(new_left, new_right, count, f)
            if count >= MAX:
              break
            last_count = count
            count = process_dir(new_left, new_right, filter, f, count, identical)
            if count - last_count > 1:
              my_print(False, None, "", file = f)
            if count < MAX:
                count = process_subdir(new_left, new_right, filter, f, count, identical)
            if count >= MAX:
                break
    return(count)

# Part 4

main ptogram

In [ ]:
MAX = 300       # just so the program will not run a long time, but there is no really a limit (I ran it against a folder with 30,000 files)
DEBUG = False   # all messages go to the screen and not saved to file

def compare_or_search(auto, left_p, right_p, filter, sub, identical):
  if not auto:
    left_p, right_p, filter, sub, identical = get_info_from_user()
    if filter == None:
      return

  count = 0
  if not right_p:
      identical = True # to display the name of files found

  with open("compare.txt", 'w') as f:
      name ='Comparing' if right_p else 'Searching'
      print(Back.YELLOW, end = '')
      offset = datetime.timezone(datetime.timedelta(hours=+2))  # to get israel time and not colab time which is UTC
      my_print(True, None, f'\n{name} files on {datetime.datetime.now(offset).strftime("%d/%m/%Y %H:%M:%S")}:', file = f)
      print(Style.RESET_ALL)
      my_print(False, None, "", file = f)
      
      dispaly_dir_name(left_p, right_p, count, f)
      if left_p.is_dir():
              # files in root folder
          last_count = count
          count = process_dir(left_p, right_p, filter, f, count, identical)
          if count - last_count > 1:
            my_print(False, None, "", file = f)
          if count < MAX:
            if sub:
                # files in sub directories
              count = process_subdir(left_p, right_p, filter, f, count, identical)

            # now check for non existed files in folder1 compared to folder 2
          if count < MAX and right_p:
            last_count = count
            count = process_dir(right_p, left_p, filter, f, count, identical = 2)
            if count - last_count > 1:
              my_print(False, None, "", file = f)
            if count < MAX:
              if sub:
                count = process_subdir(right_p, left_p, filter, f, count, identical = 2)
      else:  # compare files not folders
          comp_file(left_p, right_p, f, identical)
          
  name ='compared' if right_p else 'searched'
  if not DEBUG:
      print()
  print(f'Done! - {name} {count} files and folders')
  if right_p:        
      if not DEBUG:
          print(f'results, including identical files, can be found in {Path.cwd() / "compare.txt"}')     
  else:
      print(f'Results can be found in {Path.cwd() / "compare.txt"}')     

# Main program

Requires user input

In [ ]:
left_p, right_p, filter_, sub, identical = None, None, None, None, None
auto = False    # tell the program to get user input
#compare_or_search(auto, left_p, right_p, filter_, sub, identical)

#Automation

To automatically run the program - without user entering data, so can be run over and over for testing, uncomment the call to compare_or_search() below

In [ ]:
# This part sets paramters for running the program without the need for user input  
  
  #test data of the folders created as a sample
left_p = Path.cwd() / 'folder1'
right_p = Path.cwd() / 'folder2'
#right_p = None (to test search only)
filter = '*'
sub = True
identical = True

f"""
  #test data on local machine     
left_p = Path('m:/apps/')
right_p = Path('D:/Dropbox/DriveD/apps/')
right_p = None
filter = '*'
sub = True
identical = False
"""

auto = True
#compare_or_search(auto, left_p, right_p, filter, sub, identical)

#Creates test data

2 folders under the current folder (cwd), named folder1 and folder2

In each folder, N1 sub-directories, and in each sub directory random number of files, up to N2 files

Then in folder2 N3 files will be deleted randomely and N4 files will be changed, also randomlly

In [ ]:
import shutil
#from random import seed
from random import randint
from random import choice

def remove_dir(folder):
  if folder.exists():
    for subfolder in [folder]:
      try: 
        shutil.rmtree(str(subfolder))
      except:
        pass

def create_test_data(N1 = 10, N2 = 10, N3 = 10, N4 = 10):
  try:    # in case the part of loading lib is not run before this part
    Path.cwd()
  except:
    from pathlib import Path
    import os
    import datetime as dt

  folder1 = Path.cwd() / 'folder1'
  folder2 = Path.cwd() / 'folder2'

  for f in folder1, folder2:
    remove_dir(f)

  with open("test_data.txt", 'w') as f:
    offset = datetime.timezone(datetime.timedelta(hours=+2))  # to get israel time and not colab time which is UTC
    print(f'Log of test data created on {datetime.datetime.now(offset).strftime("%d/%m/%Y %H:%M:%S")}:\n', file = f)

    folder1.mkdir()
    print('Created folder [' + str(folder1) + ']', file = f)
    for i in range(N1 + 1):
      if i > 0:
        sub = folder1 / ('sub' + str(i - 1))
        sub.mkdir()
        print('Created sub folder [' + str(sub) + ']', file = f)
          # generate random integer values
          # seed random number generator
      #seed(i)
          # generate some integers
      j = randint(0, N2)
      for k in range(j):
        if i == 0:
          file = folder1 / ('file' + str(k))
          if k % 2 == 1:
            file = folder1 / ('file' + str(k) + '.txt')
        elif k % 2 == 0:
          file = sub / ('file' + str(k) + '.ext')
        else:
          file = sub / ('file' + str(k))
        file.touch()
        print('Created file [' + str(file) + ']', file = f)

    # copy folder1 to folder2
    shutil.copytree(src=folder1, dst=folder2)
    print('\nCopy all to [' + str(folder2) + ']\n', file = f)

          # create a list of the files generated
    files = [dirpath  + '/' + filename for dirpath, dirnames, filenames in os.walk(folder2) for filename in filenames]
  #  seed(N3 * 3)
          # make choices from the sequence to delete some files
    for _ in range(N3):
      selection = choice(files)
      if Path(selection).exists():  # in case was not removed already (random numbers can repeat themselves)
        Path(selection).unlink()
        print('\tDeleted file [' + str(selection) + ']', file = f)
    print("", file = f)      

          # create a list of the files generated in folder 1
    files = [dirpath  + '/' + filename for dirpath, dirnames, filenames in os.walk(folder1) for filename in filenames]
  #  seed(N3 * 5)
          # make choices from the sequence to delete some files
    for _ in range(max(1, N3 - 4)):
      selection = choice(files)
      if Path(selection).exists():  # in case was not removed already (random numbers can repeat themselves)
          # need also to be sure that exists in the other folder because if deleted on both will not be included in the compare
        fname = str(selection).replace('folder1', 'folder2')
        if Path(fname).exists():
          Path(selection).unlink()
          print('\tDeleted file [' + str(selection) + ']', file = f)
    print("", file = f)

          # create a list of the files after deletion (so not to modify and it will be deleted)
    files = [dirpath  + '/' + filename for dirpath, dirnames, filenames in os.walk(folder2) for filename in filenames]
  #  seed(N4 * 4)
          # make choices from the sequence to change some files
    for _ in range(N4):
      selection = choice(files)
              # need also to be sure that exists in the other folder because if delted no need to modify
      fname = str(selection).replace('folder2', 'folder1')
      if Path(fname).exists():
        Path(selection).write_text('hello!')
        print('\t\tModified file [' + str(selection) + ']', file = f)

  # to create the test data uncomment the next line
#create_test_data()

#QA

Run this to automatically test the program with the random test data created.

These are the steps done by the program:

1.   Creates the test data - log is saved in test_data.txt
2.   Run the program against the folders created by the test data - log created is saved in comared.txt
3.   Compare the 2 logs to see if they match

To make it works uncomment below the call to run_QA().

Note that to keep the test data fixed, comment below the call to create_test_data() 
create_test_data() and compare_or_search( which are commented so that Run All does not run it unintetionally

In [ ]:
import re

def run_QA():
  create_test_data()  # step 1

    # set the user input
  left_p = Path.cwd() / 'folder1'
  right_p = Path.cwd() / 'folder2'
  filter = '*'
  sub = True
  identical = True
  auto = True

  compare_or_search(auto, left_p, right_p, filter, sub, identical)  # step 2

    # step 3: compared the 2 logs
  print('\nResult of QA test:\n')

  with open("test_data.txt") as f:
    test = f.read()
  deleted_list = sorted(re.findall(r'Deleted file \[(\S+)\]', test))
    # modified list might contain duplicates since random numbers can repeat themselves
  modified_list = sorted(set([filename.replace('folder2', 'folder1') for filename in re.findall(r'Modified file \[(\S+)\]', test)]))
  #print(len(modified_list), modified_list)

  with open("compare.txt") as f:
    comp = f.read()
  not_exists_list = re.findall(r'\[(\S+)\] does not exist', comp)
  not_exists_list = sorted([f.replace('folder2', 'folder1') 
                              if 'folder2' in f 
                              else f.replace('folder1', 'folder2') 
                            for f in not_exists_list])
  not_identical_list = sorted(re.findall(r'\[(\S+)\] not identical', comp))
  #print(len(not_identical_list), not_identical_list)

  identical = True
  if deleted_list == not_exists_list:
    print('Check for "not existed" passed')
  else:
    identical = False
    for filename in deleted_list:
      if filename not in not_exists_list:
        print(f"{filename.replace('folder1', 'folder2')} deleted in test but not in not_existed list")
    for filename in not_exists_list:
      if filename not in deleted_list:
        print(f"{filename} not deleted in test but is in not_existed list")

  if modified_list == not_identical_list:
    print('Check for "not identical" passed')
  else:
    identical = False
    for filename in modified_list:
      if filename not in not_identical_list:
        print(f"{filename.replace('folder1', 'folder2')} modified in test but not in not_identical list")
    for filename in not_identical_list:
      if filename not in modified_list:
        print(f"{filename} not modified in test but is in not_identical list")

  if identical:
    print('Check for "identical" passed')

  # to  run the test uncomment the next line
#runQA()